In [11]:
from PIL import Image
from pathlib import Path
import numpy as np
import cv2
import os
from imagehash import phash
import datetime
from tqdm.notebook import tqdm

def process_images(directory, log_file_path, sample_precision):
    """
    处理图像文件，计算哈希值并筛选出唯一的图像文件路径列表。

    参数：
        directory (str): 包含图像文件的目录路径。
        log_file_path (str): 日志文件的路径。
        sample_precision (int): 图片采样精度，即图片缩放尺寸。

    返回：
        tuple: 哈希字典、唯一图像文件路径列表和处理失败的文件列表。
    """
    # 初始化变量
    hashes = {}
    unique_images = []
    failed_files = []
    
    # 获取目录中的所有文件列表
    directory_path = Path(directory)
    total_files = sum(1 for item in directory_path.iterdir() if item.is_file())

    # 记录日志：开始处理图像文件
    print("开始计算哈希，总文件数：{}".format(total_files))
    log2file(log_file_path, "总文件数：{}".format(total_files))
    files = directory_path.glob("*")

    # 遍历目录中的文件
    for file_path in tqdm(files, total=total_files, desc='Processing'):
        # 判断文件类型，仅处理jpg和png文件
        if not file_path.suffix.lower() in (".jpg", ".png"):
            continue
        try:
            with open(file_path, 'rb') as f:
                img_bytes = np.fromfile(f, dtype=np.uint8)
                img = cv2.imdecode(img_bytes, cv2.IMREAD_GRAYSCALE)
                
            if img is None:
                err_info = "警告: 无法读取图像 {}, 跳过.".format(file_path)
                print(err_info)
                failed_files.append(err_info)
                continue
            
            img_pil = Image.open(file_path)
            img_size = img_pil.size
            
            img_pil_gray = Image.fromarray(img)
            img_hash = phash(img_pil_gray.resize((sample_precision, sample_precision)))
            
            if img_hash not in hashes:
                hashes[img_hash] = {"paths": [], "sizes": []}
            hashes[img_hash]["paths"].append(file_path)
            hashes[img_hash]["sizes"].append(img_size)
            
            unique_images.append(file_path)
        except Exception as e:
            # 记录处理失败的文件及错误信息
            err_info = "错误: 处理 {} 时出错: {}".format(file_path, e)
            print(err_info)
            failed_files.append(err_info)
        
    # 返回哈希字典、唯一图像文件路径列表和处理失败的文件列表
    return hashes, unique_images, failed_files


def remove_duplicate_images(directory, remove_from_disk=False, sample_precision=8):
    """
    删除重复的图像文件。

    参数：
        directory (str): 包含图像文件的目录路径。
        remove_from_disk (bool): 是否从磁盘中删除重复文件，默认为 False。
        sample_precision (int): 图片采样精度，即图片缩放尺寸。

    返回：
        list: 唯一图像文件路径列表。
    """
    # 获取当前工作目录
    current_dir = os.getcwd()
    # 定义日志文件路径
    log_file_path = os.path.join(current_dir, "log_{}.txt".format(datetime.datetime.now().strftime("%Y%m%d%H%M%S")))

    info = "开始处理图片去重，目录：{directory}，是否删除重复文件：{remove_from_disk}".format(directory=directory, remove_from_disk=remove_from_disk)
    print(info)
    log2file(log_file_path, info)

    # 检查目录是否存在
    if not os.path.isdir(directory):
        err_info = "错误: 目录 {} 不存在。".format(directory)
        print(err_info)
        if log_file_path:  # 确保log_file_path已被定义
            log2file(log_file_path, err_info)
        return []

    # 获取目录中的文件列表
    files = os.listdir(directory)
    
    # 检查目录是否为空或没有图片文件
    if not files or all(not f.lower().endswith(('.jpg', '.png')) for f in files):
        err_info = "警告: 目录 {} 是空的或不包含任何图片文件。".format(directory)
        print(err_info)
        if log_file_path:
            log2file(log_file_path, err_info)
        return []

    # 调用处理图像文件的函数，并传递采样精度参数
    hashes, unique_images, failed_files = process_images(directory, log_file_path, sample_precision)
    error_files = []  # 用于记录删除重复文件时出现的错误的文件
    
    # 初始化重复组计数器
    duplicate_group_count = 0
    
    # 遍历哈希字典
    for hash_value, data in hashes.items():
        # 如果有重复的图像
        if len(data["paths"]) > 1:  
            duplicate_group_count += 1
            info ="第 {} 组重复图片，共 {} 张，哈希值: {}".format(duplicate_group_count, len(data['paths']), hash_value)
            print(info)
            log2file(log_file_path, info)
            
            # 找到尺寸最大的图像文件
            max_size_index = data["sizes"].index(max(data["sizes"], key=lambda x: x[0]*x[1]))
            largest_img_path, largest_img_size = data["paths"][max_size_index], data["sizes"][max_size_index]
            info = "{} w:{} h:{} - 保留（尺寸最大）".format(largest_img_path, largest_img_size[0], largest_img_size[1])
            print(info)
            log2file(log_file_path, info)
            
            # 遍历除尺寸最大的图像文件外的其他文件
            for idx, (path, size) in enumerate(zip(data["paths"], data["sizes"]), start=1):
                if idx != max_size_index + 1:
                    info = "{} w:{} h:{} - 标记删除".format(path, size[0], size[1])
                    print(info)
                    log2file(log_file_path, info)
                    unique_images.remove(path)
                    if not remove_from_disk: continue
                    try:
                        os.remove(path)
                    except Exception as e:
                        # 记录删除文件时出现的错误
                        err_info = "错误: 删除文件 {} 时出错: {}".format(path, e)
                        print(err_info)
                        error_files.append(err_info)

    info = "总重复组数：{}".format(duplicate_group_count)
    print(info)
    log2file(log_file_path, info)

    info = "去重后数量: {}".format(len(unique_images))
    print(info)
    log2file(log_file_path, info)
    
    # 如果存在处理失败或删除文件错误，记录到日志文件中
    if failed_files or error_files:
        print("\n总共失败或异常文件数:", len(failed_files) + len(error_files))
        for error_info in failed_files + error_files:
            print(error_info)
            log2file(log_file_path, error_info)
    else:
        print("\n所有文件均已成功处理，无任何失败或异常情况。")
    print("详细信息见 {}".format(log_file_path))
    
    # 返回唯一图像文件路径列表
    return unique_images


def log2file(log_file_path, message):
    """
    记录消息到日志文件。

    参数：
        log_file_path (str): 日志文件的路径。
        message (str): 要记录的消息。
    """
    # 如果文件不存在，就创建一个新文件
    if not os.path.exists(log_file_path):
        with open(log_file_path, "w"):
            pass
    # 追加写入错误信息到文件
    with open(log_file_path, "a") as f:
        f.write(message + "\n")


In [12]:
# 调用函数并接收返回的去重后文件列表
unique_images_list = remove_duplicate_images("F:\\pic\\zhytest", False, 8)

开始处理图片去重，目录：F:\pic\zhytest，是否删除重复文件：False
开始计算哈希，总文件数：52


Processing:   0%|          | 0/52 [00:00<?, ?it/s]

第 1 组重复图片，共 2 张，哈希值: e52192968c4d9bbb
F:\pic\zhytest\e0951f8ccda0475c90ec647f63168330_th.jpg w:1242 h:1880 - 保留（尺寸最大）
F:\pic\zhytest\e0951f8ccda0475c90ec647f63168330_thcopy.jpg w:1242 h:1880 - 标记删除
第 2 组重复图片，共 3 张，哈希值: f939f023467eb049
F:\pic\zhytest\551433992CB7682A291A666534355A22.jpg w:2445 h:3264 - 保留（尺寸最大）
F:\pic\zhytest\551433992CB7682A291A666534355A22copy2.jpg w:2445 h:3264 - 标记删除
F:\pic\zhytest\551433992CB7682A291A666534355A22copy.jpg w:2445 h:3264 - 标记删除
总重复组数：2
去重后数量: 49

所有文件均已成功处理，无任何失败或异常情况。
详细信息见 e:\dev\face_image_preprocessing\log_20240505193453.txt
